# <font color = cyan> NEED TO FIGURE OUT WAY TO GRAB ALL FILES USING NOTEBOOK</font>

# DSCI 550 Assignment 1
### Sunday 03/14/21

#### Group Members
- Olivia Fryt <fryt@usc.edu> 
- Katie Chak <chakw@usc.edu>
- Madeleine Thompson <mjt43250@usc.edu> 
- Sarah Pursley <spursley@usc.edu> 
- Amber Yu <qiuyunyu@usc.edu> 
- Claudia Winarko <cpwinark@usc.edu> 

## 1. Download/Install Apache Tika
1. Start GeoTopicParser-enabled Tika Server:
2.  Run a GeoTopicParser-enabled Apache Tika Server on localhost:9998. 
3. Install python tika package:
> `pip install tika`

---
<font color=red>
Containerized in my repo: https://github.com/frytoli/geotopic-parser-enabled-tika-docker
docker run -d -p 127.0.0.1:9998:9998 fryto/gtp-tika:latest</font>


## 2. Download/Install D3.js
- Package: http://d3js.org/
- Visual gallery wiki: https://github.com/mbostock/d3/wiki/Tutorials


## 3. Download Nigerian Price Fraud Email Dataset¶
Source: https://www.kaggle.com/rtatman/fraudulent-email-corpus <br>
**Local Paths: ./data/fraudulent_emails.txt, ./data/fraudulent_emails_copy.txt**

In [ ]:
import os
import sys
import json

infile = 'fraudulent_emails_copy.txt'
path_to_infile = os.path.join(os.getcwd(), 'data', infile)

outfile = 'fraudulent_emails_olivia.json'
path_to_outfile = os.path.join(os.getcwd(), 'data', outfile)

# Prepare method to save a checkpoint of the data to the "outfile"
import json

def save_to_outfile(data):
    with open(path_to_outfile, 'w') as outjson:
        json.dump(data, outjson, indent=2)
    print(f'Data saved to {path_to_outfile}')

### 4.1. "Convert" messages to json with Tika
**NOTE:**
Use of the Tika CLI yields different (and better) results than directly using the API to parse files. I can't figure out why this is. The Tika python CLI "parse" command uses the parseAndSave() method (https://github.com/chrismattmann/tika-python/blob/d692c0ffa6b85d099019de9b94888fb4c2a48040/tika/tika.py#L261) that in turn sends data to the parse1() method (https://github.com/chrismattmann/tika-python/blob/d692c0ffa6b85d099019de9b94888fb4c2a48040/tika/tika.py#L310) which interfaces with Tika to complete parsing. Interestingly enough, the from_file() parsing method in the Tika python library (https://github.com/chrismattmann/tika-python/blob/d692c0ffa6b85d099019de9b94888fb4c2a48040/tika/parser.py#L23) also passes data through the parse1() method to Tika. The only main difference between parseAndSave() and from_file() is the way the output data is handled and returned: the parseAndSave() method simply writes the item at index 1 in the returned json on a new line while the from_file() method sends the data returned from Tika to the _parse() method (https://github.com/chrismattmann/tika-python/blob/d692c0ffa6b85d099019de9b94888fb4c2a48040/tika/parser.py#L68) for additional parsing

In [ ]:
import tika    # import tika to parse files
from tika import tika

In [ ]:
# load & parse json file using tika parser
parsed = json.loads(tika.parse1('all', path_to_infile)[1])

# Ignore the first returned item, as it is associated with the infile
messages = parsed[1:]

# Save a checkpoint
save_to_outfile(messages)

### 4.2. Ensure each message is a separate json entity
** Addition of new key embedded_resources

In [ ]:
# Find and attribute message attachments (these are parsed as separate json blobs)
#   * Attachment (embedded resource) metadata is stored in the json object(s) directly before the main message object in the list returned from Tika
resource_indicies = []
resources = []
count = 0

for message in messages:
    # If at least one resource needs to be added...
    if len(resources) > 0:
        
        # If there seems to be another resource attached to the same message, append it to resources, save item index, and continue
        if message['X-TIKA:embedded_depth'] != '1':
            resource_indicies.append(count)
            resources.append(message)
            
        # Else if the current message is the main message (aka it is not an embedded message)...
        elif message['X-TIKA:embedded_depth'] == '1':
            # Iterate over the resources...
            for resource in resources:
                # Assume the message (as long as it's immediately after the attachment(s)) is associated with the attachment(s)
                if 'embedded_resources' not in message:
                    message['embedded_resources'] = [resource]
                else:
                    message['embedded_resources'].append(resource)
            # Reset
            resources = []
        else:
            print('[!] Houston we have a problem')
            # Reset
            resources = []
            
    # otherwise, check if the current message is an embedded resource
    elif message['X-TIKA:embedded_depth'] != '1':
        resource_indicies.append(count)
        resources = [message]
    
    count += 1    # increment the counter

# Remove messages that are embedded resources -- best practice is not to manipulate a list while iterating over it
delta = 0
for index in resource_indicies:
    del messages[index-delta]
    delta +=1

print(f'{len(resource_indicies)} embedded resources attributed to {len(messages)} total messages')

# save a checkpoint
save_to_outfile(messages)

### <font color=red>**If we need to see every fraud email content**</font>

In [ ]:
for message in messages:
    if "friend" in message["X-TIKA:content"]:
        print(message["X-TIKA:content"].lstrip().strip())

## 5. Dataset Enrichment with Attack Type and Attacker Stylometrics
---

### Pre-Analysis: Message Contents Language Identification
- Before analysis, use Tika to identify the language of each message's body

#### **Import Standard Python Packages**

In [ ]:
# os and sys already called
#!{sys.executable} -m pip install ipinfo
import requests
import re
import numpy as np
import pandas as pd

### <font color=red>**Main Notebook**</font>

In [ ]:
# Method for communicating with Tika Server
def languagedetect_string(string):
    r = requests.put('http://localhost:9998/language/string', data=string)
    if r.status_code == 200:
        return r.text


whitespace_ptrn = re.compile(r'\s')
langs = dict()

# Iterate over messages
for message in messages:
    # Normalize all whitespace in message body
    norm_content = whitespace_ptrn.sub(' ', message['X-TIKA:content']).lower().encode('utf-8')
    # Evaluate language with Tika
    lang = languagedetect_string(norm_content)
    if lang in langs:
        langs[lang] += 1
    else:
        langs[lang] = 1
    # Save language
    message['TIKA-language'] = lang

# Output the languages and counts
for key, value in langs.items():
    print(f'{key} : {value}')

# Save a checkpoint
save_to_outfile(messages)

### 5.a. Attack Type Classification
- Begin by generating a word cloud to gain insight into the content of the messages:

### <font color=red>**Main Notebook**</font>

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
# Join and normalize all message contents and message subjects as strings
whitespace_ptrn = re.compile(r'\s')
cloud_corpus = ' '.join(whitespace_ptrn.sub(' ', message['X-TIKA:content']).lower().strip() for message in messages)

# Generate word cloud
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = set(STOPWORDS),
    random_state = 1,
    collocations = False
).generate(str(cloud_corpus))

# Plot
fig = plt.figure(1, figsize=(40, 30))
plt.axis('off')
fig.suptitle('Fig 1. Message Content', fontsize=40)
fig.subplots_adjust(top=1.06)

plt.imshow(wordcloud)
plt.show()

Categorize the messages into the following social engineering types by 
1.  Scanning a compiled set of keywords 
2.  Recognizing suspicious attachments/resources (in other words, messages with an attachment are considered to be of SE type "malware"):

**SE Types:** Reconnaissance, Social Engineering, Malware, Phishing

**NOTE:** Intelligent categorization into these four categories is not possible without training data (must be supervised).

In [ ]:
en_keywords = {
    'recon': ['reply back', 'click here', 'confidential', 'contact', 'response', 'private', 'personal', 'information'],
    'social_engineering': ['your friend', 'comrade', 'urgent', 'need help', 'threat', 'act now', '$', 'money', 'transaction', 'give', 'assist', 'business', 'fund', 'help', 'unclaimed', 'please', 'died'],
    'malware': ['click here', 'http'],
    'phishing': ['social security', 'ssn', 'date of birth', 'birthday', 'dob', 'account number', 'bank', 'beneficiary', 'password', 'phone number', 'fax number', 'company address']
}
fr_keywords = {} # TBD!
es_keywords = {} # TBD!

# Keep count of keywords to get more insight/clean keywords
kw_count = {kw:0 for kw in keywords}

# Tag messages
for message in messages:
    message['se_tags'] = []
    # If message is in French, use French keywords
    if message['TIKA-language'] == 'fr':
        keywords = fr_keywords
    # Else if message is in Spanish, use Spanish keywords
    elif message['TIKA-language'] == 'es':
        keywords = es_keywords
    # Otherwise, use English keywords (manual review shows that all other languages also have an English part AND I don't know anything about the other languages)
    else:
        keywords = en_keywords
    # Find keywords in message body
    for kw_type in keywords:
        for kw in keywords[kw_type]:
            if kw in message['X-TIKA:content']:
                if kw_type not in message['se_tags']:
                    message['se_tags'].append(kw_type)
                    kw_count[kw_type] += 1
    # If the message contains at least one attachment, classify it as "malware"
    if 'embedded_resources' in message:
        if 'malware' not in message['se_tags']:
            message['se_tags'].append('malware')
            kw_count['malware'] += 1
        

print('Count of messages by tag:')
for kw in sorted(kw_count, key=kw_count.get, reverse=True):
    print(f'  {kw}: {kw_count[kw]}')

# Save a checkpoint
save_to_outfile(messages)

## 5.b. Attacker Stylometric Enrichment

### i. Find attacker title
**NOTE:** Named entity recognition with NLTK does not notice titles without personalization. Therefore, fuzzy string matching was utilized.

### <font color=red>**Main Notebook**</font>

In [ ]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [ ]:
# Define a set of sender titles
titles = ['mr', 'mrs', 'miss', 'ms', 'madame', 'honourable', 'lady', 'senator', 'governor', 'hon', 'col', 'colonel',
          'president', 'partner', 'senior', 'ceo', 'executive', 'officer', 'chief', 'maitre', 'dr', 'doctor', 'barr',
          'barrister', 'esq', 'esquire', 'manager', 'executive', 'sister', 'brother', 'captain', 'capt', 'head', 'lead',
          'director', 'prince', 'princess', 'king', 'queen', 'chairman', 'judge', 'engr', 'engineer', 'minister']
abrv_titles = {
    'dr': 'doctor',
    'barr': 'barrister',
    'esq': 'esquire',
    'capt': 'captain',
    'engr': 'engineer',
    'ceo': 'chief executive officer',
    'hon': 'honourable',
    'col': 'colonel'
}
# Note that CEO is a bit strange, as the title can be set from three different titles
#  ceo = all(['chief', 'executive', 'officer'])

# Iterate over messages
for message in messages:
    # Create "author_titles" key-value pair
    message['author_titles'] = []
    temp_titles = set()
    # Fuzzy match author name, if applicable
    if 'Author' in message:
        # If applicable, pick last author in author string to fuzzy match on (this item seems to have been parsed more cleanly by Tika)
        if len(message['Author']) > 1:
            author = message['Author'][-1].lower()
        else:
            author = message['Author'][0].lower()
        # Split author string by space, period, comma, and dash, and normalize (lowercase)
        split_author = re.split('[ .,-]', author.lower())
        # Iterate over titles
        for title in titles:
            # Iterate over split parts of author
            for part in split_author:
                # Fuzzy match the parse parts of the author string with each title
                ratio = fuzz.ratio(title, part)
                if ratio > 86: # 86 seems to be the threshold at which "mrs" != "mr" and "prince" != "princess"
                    # Add title to set
                    temp_titles.add(title)
    # If not title was found in the author name or an author name does not exist, separate message content at last comma, split, and fuzzy match
    if 'Author' not in message or len(temp_titles) == 0:
        indexof_last_comma = message['X-TIKA:content'].rfind(',')
        endof_body = message['X-TIKA:content'][indexof_last_comma+1:]
        # Split string by space, period, comma, and dash, and normalize (lowercase)
        split_endof_body = re.split('[\s.,\-()]', endof_body.lower())
        # Iterate over titles
        for title in titles:
            # Iterate over split parts of body
            for part in split_endof_body:
                # Fuzzy match the parse parts of the author string with each title
                ratio = fuzz.ratio(title, part)
                if ratio > 86: # 86 seems to be the threshold at which "mrs" != "mr" and "prince" != "princess"
                    # Add title to set
                    temp_titles.add(title)
    # Make titles consistent (handle abbreviations)
    # First a foremost, lets see if author is a ceo ** Note the comment above
    ceo_check = all(item in list(temp_titles) for item in ['chief', 'executive', 'officer'])
    if ceo_check:
        temp_titles.add('ceo')
        temp_titles.remove('chief')
        temp_titles.remove('executive')
        temp_titles.remove('officer')
    # Iterate over titles and add to message blob
    for title in temp_titles:
        try:
            message['author_titles'].append(abrv_titles[title])
        except:
            message['author_titles'].append(title)

# Save a checkpoint
save_to_outfile(messages)

### ii.  Attribute a level of urgency ("urgency score") to each message

### <font color=red>**Main Notebook**</font>

In [ ]:
 Synonyms/antonyms of "urgent" from thesaurus.com: https://www.thesaurus.com/browse/urgent
#  Some words removed to avoid confusion, mainly in regards to sender titles i.e. "chief" and "principal"
#  Some words added after manual review of messages

urgency_ratings = {
    2: ['urgency', 'urgent', 'compelling', 'compel', 'critical', 'crucial', 'demanding', 'essential', 'immediate',
        'imperative', 'important', 'indispensable', 'necessary', 'persuasive', 'pressing', 'serious', 'vital', 'weighty',
        'as soon as possible', 'may not exceed', 'now'],
    1: ['burning', 'called-for', 'clamant', 'clamorous', 'crying', 'demanded', 'driving', 'exigent', 'foremost', 'heavy',
        'hurry-up', 'impelling', 'importunate', 'insistent', 'instant', 'leading', 'life and death', 'momentous', 'paramount',
        'primary', 'required', 'salient', 'top-priority', 'touch and go', 'touchy', 'wanted', 'quick', 'earliest response',
        'at once', 'right now'],
    -1: ['easy', 'facile', 'inessential', 'insignificant', 'light', 'minor', 'needless', 'nonessential', 'optional',
         'secondary', 'small', 'trivial', 'uncritical', 'unimportant', 'unnecessary', 'unsubstantial', 'voluntary',
         'eventually', 'later']
}

scores = []

# Iterate over messages
for message in messages:
    # Iterate over words in urgency ratings, see if they exist in the message's body, and compile a score accordingly
    message['urgency_score'] = 0
    for score in urgency_ratings:
        for word in urgency_ratings[score]:
            # If the word is found in the message's body content, add the associated score (note that antonyms take away from the overall urgency score)
            if word in message['X-TIKA:content'].lower():
                message['urgency_score'] += score
    scores.append(message['urgency_score'])

# Output max, avg, min of scores
print(f'Max "urgency score": {max(scores)}')
print(f'Avg "urgency score": {sum(scores)/len(scores)}')
print(f'Min "urgency score": {min(scores)}')

# Save a checkpoint
save_to_outfile(messages)

### iii. Find the date/time of the email attack
Convert message creation timestamps to timezone-aware (if possible) ISO 8601 format strings

In [ ]:
import dateutil
from dateutil import parser
from pytz import timezone

### <font color=red>**Main Notebook**</font>

In [ ]:
# Compile regex patterns
whitespace_ptrn = re.compile(r'\s')
mboxfrom_date_ptrn = re.compile(r'(Mon|Tue|Wed|Thu|Fri|Sat|Sun) {1,2}(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)( [1-9][0-9]|  [1-9]) [0-9]{2}:[0-9]{2}:[0-9]{2} [1-2][0-9]{3}')
date_ptrn = re.compile(r'(Mon|Tue|Wed|Thu|Fri|Sat|Sun), [1-9][0-9]{0,1} (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) [1-2][0-9]{3} [0-9]{2}:[0-9]{2}:[0-9]{2} -[0-9]{4} \([A-Z]{3}\)')

nodate = []

# Iterate over messages
for message in messages:
    # Keep track of datetime object itself
    dt = None
    # If "Creation-Date" in message record, convert value directly to isoformat
    if 'Creation-Date' in message:
        # Convert "Creation-Date" to ISO 8601
        dt = parser.isoparse(message['Creation-Date'])
    # Otherwise, if "MboxParser-from" in message record, convert this value to isoformat AND use previously discovered geolocation for timezone
    elif 'MboxParser-from' in message:
        # Prioritize IP address timezone, then Tika GeoTopicParser locations
        if 'IPInfo-data' in message:
            # Normalize value to parse
            mboxparser_from = whitespace_ptrn.sub(' ', message['MboxParser-from'])
            # Parse date from value
            dtstr = mboxfrom_date_ptrn.search(mboxparser_from)
            if dtstr:
                dt = parser.parse(dtstr.group(0))
                # Update with timezone aware value
                tz = timezone(message['IPInfo-data']['timezone'])
                dt = tz.localize(dt)
        # Else parse date from 'MboxParser-return-path' or from 'MboxParser-x-sieve'/'MboxParser-from' if possible
        elif 'MboxParser-return-path' or 'MboxParser-x-sieve' in message:
            # 'MboxParser-return-path' logic
            if 'MboxParser-return-path' in message:
                # Handle potential types of 'MboxParser-return-path' value
                if type(message['MboxParser-return-path']) == str:
                    # Look for date pattern in string
                    dtstr = date_ptrn.search(message['MboxParser-return-path'])
                    # If a match was found, parse it to datetime
                    if dtstr:
                        # Parse out date
                        dt = parser.parse(dtstr.group(0))
                        # Acknowledge that timestamp was found
                        timestamp_found = True
                elif type(message['MboxParser-return-path']) == list:
                    # Iterate over items
                    for path in message['MboxParser-return-path']:
                        # Look for date pattern in string
                        dtstr = date_ptrn.search(path)
                        # If a match was found, parse it to datetime
                        if dtstr:
                            # Parse out date
                            dt = parser.parse(dtstr.group(0))
                            # Break out of loop
                            break
                else:
                    print('[!] Houston we have a problem')
            # 'MboxParser-x-sieve'/'MboxParser-from' logic
            if not dt and 'MboxParser-x-sieve' in message:
                # Look for date pattern in string
                dtstr = date_ptrn.search(message['MboxParser-x-sieve'])
                #
                # It seems that the value of 'MboxParser-from' is typically just a couple minutes different than the value from
                # 'MboxParser-x-sieve'. We'll check that this is the case, and if it is we'll use the timezone from 'MboxParser-x-sieve'
                # and the datetime value from 'MboxParser-from'
                #
                # If a match was found...
                if dtstr:
                    # Extract timezone from dtstr
                    tzstr = dtstr.group(0).split(' ')[-2]
                    # Normalize 'MboxParser-from' value to parse
                    mboxparser_from = whitespace_ptrn.sub(' ', message['MboxParser-from'])
                    # Parse date from value
                    dtstr = mboxfrom_date_ptrn.search(mboxparser_from)
                    # Combine datetime and timezone strings, and parse to datetime object
                    dt = parser.parse(f'{dtstr.group(0)} {tzstr}')
    
    # If no timestamp found yet, attempt to parse timezone unaware timestamp
    if not dt:
        # Normalize value to parse
        mboxparser_from = whitespace_ptrn.sub(' ', message['MboxParser-from'])
        # Parse date from value
        dtstr = mboxfrom_date_ptrn.search(mboxparser_from)
        if dtstr:
            dt = parser.parse(dtstr.group(0))

    # Store timestamp in message as ISO 8601 string
    message['timestamp'] = dt.isoformat()

# Save a checkpoint
save_to_outfile(messages)

### <font color=red>**Claudia's Notebook**</font>

In [ ]:
nodate = []

# Iterate over messages
for message in messages:
    # If "Creation-Date" in message record, convert value directly to isoformat
    if 'Creation-Date' in message:
        # Convert "Creation-Date" to ISO 8601 and store in *** new key "timestamp" ***
        message['timestamp'] = parser.isoparse(message['Creation-Date']).isoformat()
    # Otherwise, if "MboxParser-from" in message record, convert this value to isoformat AND use previously discovered geolocation for timezone
    elif 'MboxParser-from' in message:
        # Prioritize IP address timezone, then Tika GeoTopicParser locations
        
    else:
        print('[!] Houston we have a problem')

### iv. Categorize attacker's offering
Offerrings: percentage of money, lump-sum of money, religious benefit, investment management opportunity, product for sale, loan, bank account, social media invitation (invitation from facebox), wanted (sender allegedly being arrested for something)

<font color = darkblue>**Start by using k-means clustering to find natural clusters based on language to gain insight into the individual meanings behind the messages:**</font>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
def see_clusters(corpus, k=2):
    # Using TfidfVectorizer and English stopwords (because majority of messages are in English)
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    
    # Create model
    model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)
    
    # Get centroids and features
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    # Iterate and output
    for i in range(k):
        print(f'Cluster {i}:')
        for ind in order_centroids[i, :10]:
            print(f'  {terms[ind]}')

In [ ]:
# First, join and normalize all message contents
whitespace_ptrn = re.compile(r'\s')
corpus = [whitespace_ptrn.sub(' ', message['X-TIKA:content']).lower().strip() for message in messages]

# Get insight from k-means clusters (mess with the value of k)
see_clusters(corpus, k=10)

### <font color = red>**Katie's Notebook**</font>
- Topic 1: asks for money for family emergency {t1:["relative","deceased","money",]}
- Topic 2: asks for money for charity {t2:["charity","friend","bless"]}
- Topic 3: asks to transfer money to Africa for foreign assistance {t3:["Africa","funds","contract","foreign"]}
- Topic 4: asks for money for business investment funds {t4:["business","investment","transaction","document"]}
- Topic 5: job offer from Tasaki-Shinju {t5:["tasaki","shinju","company","family","job"]}
- Topic 6: offering sum of money from bank in Nigeria {t6:["foreign","fund","nigeria","country"]}
- Topic 7: need help to get sums of money (will split sum) {t7:["split", "unclaimed","transfer","claim","father"]}

In [ ]:
offerings={"t1":["relative","deceased","money","family"],
           "t2":["charity","friend","bless","cheque","organization","ramitez"],
           "t3":["Africa","funds","contract","foreign"],
           "t4":["business","investment","transaction","document"],
           "t5":["tasaki","shinju","company","family","job"],
           "t6":["foreign","fund","nigeria","country"],
           "t7":["split", "unclaimed","transfer","claim","father"]}

def get_ans(topic_wrd_cnt,total_word,topic_name):
    ratio=topic_wrd_cnt/total_word
    return (ratio,topic_name)    # returns tuple of topic name and its strength

In [ ]:
for message in messages:
    # initializing topic ratios
    t1=0
    t2=0
    t3=0
    t4=0
    t5=0
    t6=0
    t7=0
    words=0
    for k,v in offerings.items():
        #print(k,v)
        email=message["X-TIKA:content"].lstrip().strip()
        token=prepare_text_for_lda(email)
        #print(token)
        for word in token:
            if word in v and k=="t1":
                t1+=1
            if word in v and k=="t2":
                t2+=1
            if word in v and k=="t3":
                t3+=1
            if word in v and k=="t4":
                t4+=1
            if word in v and k=="t5":
                t5+=1
            if word in v and k=="t6":
                t6+=1
            if word in v and k=="t7":
                t7+=1
            words+=1
    t1=get_ans(t1,words,"t1")
    t2=get_ans(t2,words,"t2")
    t3=get_ans(t3,words,"t3")
    t4=get_ans(t4,words,"t4")
    t5=get_ans(t5,words,"t5")
    t6=get_ans(t6,words,"t6")
    t7=get_ans(t7,words,"t7")
    total_list=[t1,t2,t3,t4,t5,t6,t7]
    #print(total_list)
    #print(total_list)
    #sort list in ascending order
    total_list.sort()
    
    message["offering"]=total_list[-1]
    #print(message["offering"])

### v. Identify attacker's location
#### **Get geolocation with geoNames.txt**  
- Use GeoTopicParser in Tika: https://cwiki.apache.org/confluence/display/tika/GeoTopicParser   

**NOTES:**
Was unable to get GeoTopicParser to work with Python Tika, so I put together and am using a containerized GeoTopicParser-enabled Tika Server (my repo: https://github.com/frytoli/geotopic-parser-enabled-tika-docker). 
1. Run this in the background (see step 1 above), then run code below:
2. Find geolocations mentioned in each message's content with Tika's GeoLocationParser:

### <font color=red>**Main Notebook**</font>

In [ ]:
import ipinfo

In [ ]:
# Define method for communicating with GeoTopicParser-enabled Tika Server
def geotopicparse_file(filename, filedata):
    headers = {
        'Content-Disposition': f'attachment; filename={filename}'
    }
    r = requests.put('http://localhost:9998/rmeta', headers=headers, data=filedata)
    if r.status_code == 200:
        return r.json()

    
whitespace_ptrn = re.compile(r'\s')

# Iterate over messages
for message in messages:
    # Normalize all whitespace in message body
    norm_content = whitespace_ptrn.sub(' ', message['X-TIKA:content']).encode('utf-8')
    # Parse each message content for geotopics
    parsed = geotopicparse_file('message.geot', norm_content)
    # Save extracted geolocations to message json
    message['TIKA-GeoLocationParser'] = parsed

# Save a checkpoint
save_to_outfile(messages)

<font color = DarkBlue>**Find geolocation based off of sender IP addresses using IPInfo (https://github.com/ipinfo/python) (note that IP addresses change, so this data might not be entirely accurate):**</font>

In [ ]:
# Initialize ipinfo object and handler
ipinfo_token = 'eb65e70ef00bb6' # 50,000 free requests per month
handler = ipinfo.getHandler(ipinfo_token)

# Compile regex patterns
whitespace_ptrn = re.compile(r'\s')
ip_ptrn = re.compile(r'(?<![0-9])[1-9][0-9]{0,2}\.[1-9][0-9]{0,2}\.[1-9][0-9]{0,2}\.[1-9][0-9]{0,2}(?![0-9])')

# IP Parsing method
def parse_ip(string):
    # Normalize value to parse
    wstring = whitespace_ptrn.sub(' ', string)
    # Parse date from value
    ip = ip_ptrn.search(wstring)
    return ip

# Iterate over messages
for message in messages:
    ip = None
    if 'MboxParser-x-sender-ip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-sender-ip'])
    if 'MboxParser-x-senderip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-senderip'])
    if not ip and 'MboxParser-x-origination-ip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-origination-ip'])
    if not ip and 'Message:Raw-Header:X-ORIGINATE-IP' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['Message:Raw-Header:X-ORIGINATE-IP'])
    if not ip and 'MboxParser-xoriginalsenderip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-xoriginalsenderip'])
    if not ip and 'MboxParser-x-original-ip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-original-ip'])
    if not ip and 'MboxParser-x-aol-ip' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-aol-ip'])
    if not ip and 'MboxParser-x-php-remote_addr' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-x-php-remote_addr'])
    if not ip and 'Message:Raw-Header:Message-ID' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['Message:Raw-Header:Message-ID'])
    if not ip and 'MboxParser-xparam2' in message:
        # Preprocess string and parse ip address
        ip = parse_ip(message['MboxParser-xparam2'])
    
    if ip:
        # Get ip address details from ipinfo
        details = handler.getDetails(ip.group(0).strip())
        # Retrieve and save all detail information
        message['IPInfo-data'] = details.all

# Save a checkpoint
save_to_outfile(messages)

### vi. Attacker's alleged relationship to the recipient

### <font color=red>**Main Notebook**</font>

In [ ]:
relationship={"foreign_partner":["foreign","partner","father","majesty","assistance","sister"],
             "stranger_in_need":["wife","husband","son","father","sister"],
              "friend":["friend","dear","help","assistance","money"]}


for message in messages:
    # initializing topic ratios
    t1=0
    t2=0
    t3=0

    words=0
    for k,v in relationship.items():
        #print(k,v)
        email=message["X-TIKA:content"].lstrip().strip()
        token=prepare_text_for_lda(email)
        #print(token)
        for word in token:
            if word in v and k=="foreign_partner":
                t1+=1
            if word in v and k=="stranger_in_need":
                t2+=1
            if word in v and k=="friend":
                t3+=1
        
            words+=1
    t1=t1/words
    t2=t2/words
    t3=t3/words

    total_list=[t1,t2,t3]
    #print(total_list)
    #print(total_list)
    total_list.sort()    # Sort list in ascending order
    if total_list[-1]==t1:
        message["relationship"]="foreign_partner"
    elif total_list[-1]==t2:
        message["relationship"]="stranger_in_need"
    else:
        message["relationship"]="friend"
    print(message["relationship"])

In [ ]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

### vii. Attacker email sentiment
https://github.com/USCDataScience/SentimentAnalysisParser

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
#using pre-trained NLTK sentiment analyzer 
sia = SentimentIntensityAnalyzer()
for message in messages: 
    message['Sentiment_Analysis'] = sia.polarity_scores(message['X-TIKA:content'])
    
# Save a checkpoint
save_to_outfile(messages)

### viii. Attacker language style

**1. Ratio of Misspelled Words** (https://github.com/mammothb/symspellpy)

### <font color=red>**Claudia's Notebook**</font>

In [ ]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

In [ ]:
# Initialize symspell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Find and load the dictonary resources
dictionary_path = pkg_resources.resource_filename('symspellpy', 'frequency_dictionary_en_82_765.txt')
bigram_path = pkg_resources.resource_filename('symspellpy', 'frequency_bigramdictionary_en_243_342.txt')
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# Compile regex patterns to ignore in message body contents
email_ptrn = re.compile(r'(?!_)[\w]{1,11}@(?![_\-.])[\w]{1,11}(\.[a-zA-Z]{1,10}){1,2}')
phone_ptrn = ''
addrs_ptrn = ''

# Iterate over messages and find incorrect spellings
for message in messages:
    suggestions = sym_spell.lookup_compound(message['content-body'], max_edit_distance=2, transfer_casing=True)
    print(message['content-body'])
    print('\n')
    for suggestion in suggestions:
        print(suggestion)
    break

### <font color=red>**Main Notebook**</font>

In [ ]:
#import sys
#!{sys.executable} -m pip install symspellpy
import pkg_resources
from symspellpy import SymSpell, Verbosity
#import re

# Initialize symspell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# !!! Account for other languages? I.e. French and Spanish messages
# Find and load the dictonary resources
dictionary_path = pkg_resources.resource_filename('symspellpy', 'frequency_dictionary_en_82_765.txt')
bigram_path = pkg_resources.resource_filename('symspellpy', 'frequency_bigramdictionary_en_243_342.txt')
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# Compile regex patterns to ignore in message body contents
whitespace_ptrn = re.compile(r'\s')
punctuation_ptrn = re.compile(r'[^\w\s\-]')
number_ptrn = re.compile(r'[0-9]')
letter_ptrn = re.compile(r'[a-z]')

# List of stopwords
stopwords = set(['mr', 'dr', 'mrs', 'usd', 'usdm', 'phd', 'hrm', 'ceo', 'drc', 'jk', 'mb', 'kg', 'nnpc', 'es', 'ie', 'ps', 'kabila'])

# Iterate over messages and find incorrect spellings
for message in messages:
    # Add author name to stopwords
    if 'Author' in message:
        if type(message['Author']) == list:
            author = message['Author'][-1]
        elif type(message['Author']) == str:
            author = message['Author']
        author = whitespace_ptrn.sub(' ', author)
        author = punctuation_ptrn.sub('', author).replace('_','')
        for part in author.split(' '):
            if part != '':
                stopwords.add(part.lower())
    # Count all words
    words = 0
    # Count misspellings
    misspellings = 0
    # Normalize and split message content
    content = whitespace_ptrn.sub(' ', message['X-TIKA:content'])
    content = punctuation_ptrn.sub('', content).replace('_','')
    content = number_ptrn.sub('', content)
    content = content.lower().split(' ')
    # Iterate over split content
    for word in content:
        # Ignore empty strings and words that do not have any letters (i.e. "--")
        if word and letter_ptrn.search(word):
            # Record word
            words += 1
            # Don't check spellings of stopwords or words with length <= 1
            if word not in stopwords and len(word)>1:
                suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
                # Record misspelled words
                if len(suggestions) > 1:
                    misspellings +=1
    # Save ratio of misspellings
    message['misspelled_ratio'] = misspellings/words

# Save a checkpoint
save_to_outfile(messages)

<font color=darkblue>**Misspellings Distribution**</font>

In [ ]:
misspells=[]
for message in messages:
    misspells.append(message["misspelled_ratio"])
    
mp_df=pd.DataFrame(misspells)
mp_df.describe()

In [ ]:
mnorm_df=pd.concat([mp_df,pd.DataFrame(msg_len)],axis=1)
mnorm_df.columns=["misspell_ratio","msg_size"]
mnorm_df["norm_ratio"]=mnorm_df["misspell_ratio"]/mnorm_df["msg_size"]

mnorm_df.describe()

In [ ]:
boxplot = mnorm_df.boxplot(column=["misspell_ratio"])

In [ ]:
boxplot = mnorm_df.boxplot(column=["norm_ratio"])

In [ ]:
mp_df.hist(bins=40)   # most misspellings lies between 0 and 1

#### **2. Ratio of random capitalizations**

In [ ]:
def read_file(filename):
    with open(filename, 'r') as file:
        data = file.read().replace('\n', '')
    return data

In [ ]:
str1=read_file("opinion_1.txt")
str2=read_file("academic_journal_2.txt")
str3=read_file("opinion_2.txt")
str4=read_file("opinion_4.txt")
str5=read_file("opinion_3.txt")
str6=read_file("academic_journal_3.txt")
str7=read_file("diary_4.txt")
str8=read_file("poem_1.txt")
str9=read_file("poem_2.txt")
str10=read_file("poem_3.txt")
str11=read_file("poem_4.txt")

# Create a list of strings from all txt files
strlist=[str1,str2,str3,str4,str5,str6,str7,str8,str9,str10,str11]

In [ ]:
# Counts number of capital letters and periods in a string and returns the ratio
def count_period_ratio(string):
    """Input is a string of article"""
    period_cnt=0
    capital_cnt=0
    
    for letter in string:
        if letter==".":
            period_cnt+=1
        elif letter.isupper()==True:
            capital_cnt+=1
    
    ratio=capital_cnt/period_cnt
    return ratio

In [ ]:
# Make a list of ratios from some test strs (articles, academic journals, poems, and diaries)
ratearray=[]
testarray=[]

for strs in strlist:
    ratearray.append(count_period_ratio(strs))
    
# Take a sample scam email texts to compare thresholds
for i in range (1000):
    try:
        testarray.append(count_period_ratio(messages[i]["X-TIKA:content"].strip().lstrip()))
    except ZeroDivisionError:
        pass
    continue

In [ ]:
# Output capitalization ratio summary stats from test strs array
df1=pd.DataFrame(ratearray).describe()
df1.columns=["normal_text"]
df1

In [ ]:
# Output capitalization ratio summary stats from sample scam emails array
df2=pd.DataFrame(testarray).describe()
df2.columns=["999_scam_text"]
df2

<font color=darkblue>**Visualization of capitalization ratio distribution**</font>

In [ ]:
fig, ax = plt.subplots(figsize=(8,4,))
counts, bins, patches = ax.hist(cap_df, facecolor="orange", edgecolor='gray',bins=100)
#ax.set_xticks(bins.round(2))
#plt.xticks(rotation=70)
plt.title('Distribution of capitalization ratio', fontsize=15)
plt.ylabel('Frequency', fontsize=10)
plt.xlabel('Capitalization Ratio', fontsize=10)

for patch, leftside, rightside in zip(patches, bins[:-1], bins[1:]):
    if rightside < 1:
        patch.set_facecolor("peachpuff")
    elif leftside > 3.76:
        patch.set_facecolor("deepskyblue")

<font color=darkblue>**Normalized data based on email message size**</font>

In [ ]:
msg_len=[]
for message in messages:
    msg_len.append(len(message["X-TIKA:content"].strip().lstrip().split(" ")))

norm_df=pd.concat([cap_df,pd.DataFrame(msg_len)],axis=1)
norm_df.columns=["cap_ratio","msg_size"]
norm_df["norm_ratio"]=norm_df["cap_ratio"]/norm_df["msg_size"]

boxplot1 = norm_df.boxplot(column=["cap_ratio"])
boxplot2 = norm_df.boxplot(column=["norm_ratio"])

<font color=darkblue>**Normalized Data Visualization**</font>

<font color=red>*CHECK NEW BOXPLOT OUTPUTS*</font>

In [ ]:
norm_df.describe()

In [ ]:
boxplot1 = norm_df.boxplot(column=["cap_ratio"])
boxplot2 = norm_df.boxplot(column=["norm_ratio"])

# basic plot
axs[0, 0].boxplot1(data)
axs[0, 0].set_title('Cap_Ratios')

# try axs[0, 1].boxplot2(data).set_title('Normalized_Ratios')
axs[0, 1].boxplot2(data)
axs[0, 1].set_title('Normalized_Ratios')

# Multiple box plots on one Axes
data = [boxplot1, boxplot1]
fig, axs = plt.subplots(1, 2)
ax.boxplot(data)

plt.show()

### *ix. Attacker estimated age*
- https://github.com/USCDataScience/AgePredictor

### <font color=red>**Main Notebook**</font>

### *x. Attacker IP known as Phisher?*
- https://scamalytics.com/ip

### <font color=red>**Madeleine's Notebook**</font>

<font color=darkblue>**Define functions to lookup IP address and find risk score**</font>

In [ ]:
import ipinfo
import random
from bs4 import BeautifulSoup

In [ ]:
def select_user():
    user_agents = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',	
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11.0; rv:83.0) Gecko/20100101 Firefox/83.0',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/29.0 Mobile/15E148 Safari/605.1.15',
        'Mozilla/5.0 (iPad; CPU OS 11_0_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/29.0 Mobile/15E148 Safari/605.1.15',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1'
    ]
    return random.choice(user_agents)

def lookup_ip(IP):
    url = 'https://scamalytics.com/ip/'
    d = {}
    try:
        page = requests.get(
            url + IP,
            headers = {'User-Agent': select_user()}
        )
    except:
        print("Error")
    soup = BeautifulSoup(page.content, 'html.parser')
    for text in soup.find_all('pre'):
        if 'ip' in text.string:
            new_data = json.loads(text.string)
            if new_data['ip'] == IP:
                d['risk_score'] = new_data['score']
                d['risk'] = new_data['risk']
    return d


In [ ]:
for message in messages:
    if 'IPInfo-data' in item:
        temp = item['IPInfo-data']
        append_info = lookup_ip(temp['ip'])
        if append_info != {}:
            temp['risk_score'] = append_info['risk_score']
            temp['risk'] = append_info['risk']
            print(temp['ip'], append_info)

### Export Updated JSON File with Stylometrics

In [ ]:
file = 'fraudulent_emails_features.json'
with open(file , 'w') as outfile:
    json.dump(messages, outfile)

def write_json(data, filename='/Users/madeleine/Desktop/DSCI_550/Assignment_1/Data/fraudulent_emails_update_3_11.json'): 
    with open(filename,'w') as f: 
        json.dump(data, f, indent=4)
        f.close()
    print("Data saved to " + filename.split('/')[-1])

## 6.  Analyze and Add Additional Datasets
--- 

### Dataset 1:  Global Unemployment Data
#### <font color = darkblue>MIME Top-Level: appplication<br>MIME Sub-level: Excel (.xls)</font>   
- Purpose: 
- Source: *Archived files*

#### <font color = darkblue>Unemployment New Features</font>
Geographic_NAME:
1. Unemployment by Location (geographic_name_unemployment)
2. Unemployment by Year when email was sent (geographic_year_unemployment)
3. Unemployment by Country (geographic_country_unemployment)

In [ ]:
filename = '/Users/madeleine/Desktop/DSCI_550/Assignment_1/Datasets/Global_Unemployment.xls'
sheet = 'Data'

df = pd.read_excel(filename, sheet_name= sheet)

header_row = 2
df.columns = df.iloc[header_row]
df = df.drop([0,1,header_row])

df = df.set_index('Country Name', drop=False)
df.head()

In [ ]:
def lookup_location(location, year, df=df):
    d = {}
    for i in df['Country Name']:
        if i.lower() in location.lower():
            d['geographic_name'] = location
            d['year'] = year
            d['country'] = i
            d[str(year) + '_unemployment'] = df.loc[i, year]
            d[str(year-5) + '_unemployment'] = df.loc[i, year-5]
            d[str(year+5) + '_unemployment'] = df.loc[i, year+5]
    return d

In [ ]:
for message in messages:
    if 'GeoLocationParser-data' in item:
        year = int(item['timestamp'].split('-')[0])
        GeoLocations = item['GeoLocationParser-data'][0]#just gets what we need, no need to iterate over 1 item
        keys = GeoLocations.keys()
        for key in keys:
            regex = re.compile('Optional_NAME[\d]*')
            if key == 'Geographic_NAME':
                location = GeoLocations['Geographic_NAME']
                try:
                    to_append = lookup_location(location, year, df)
                    if to_append != {}:
                        item['GeoLocationParser-data'].append({(key + "_unemployment"): to_append})
                except:
                    print("Error: " + key + " " + location)
            elif re.match(regex, key):
                location = GeoLocations[key]
                try:
                    to_append = lookup_location(location, year, df)
                    if to_append != {}:
                        item['GeoLocationParser-data'].append({(key + "_unemployment"): to_append})
                except:
                    print("Error: " + key + " " + location)

### Dataset 2:  World Bank International Debt Statistics
#### <font color = darkblue>MIME Top-Level: text<br>MIME Sub-level: csv</font>   
- Purpose: Identify economic debt indicators of (ip_address or geolocation)
- Source: *Archived files*

#### <font color = darkblue>Debt New Features</font>
1. Gross National Income (GNI) in USD
2. Total debt service (% of exports of goods, services and primary income)
3. Personal transfers and compensation of employees received in USD

In [ ]:
filename = '/Users/madeleine/Desktop/DSCI_550/Assignment_1/Datasets/IDS_CSV/IDS-DRSCountries_WLD_Data.csv'
ids_df = pd.read_csv(filename)

ids_df = ids_df.set_index('Country Name', drop=False)
ids_df.head()

In [ ]:
codes = {'GNI_USD':'NY.GNP.MKTP.CD', 
         'Debt_Service_%':'DT.TDS.DECT.EX.ZS', 
         'Employee_Compensation_USD':'BX.TRF.PWKR.CD.DT'}

In [ ]:
def lookup_loc(location, year, df=ids_df):
    countries = set(ids_df['Country Name'])
    countries.remove(np.nan)
    countries.remove('Last Updated: 10/16/2020')
    countries.remove('Least developed countries: UN classification')
    countries.remove('Data from database: International Debt Statistics')


    d = {}
    
    if location == 'Federal Republic of Nigeria':
        d['country'] = 'Nigeria'
        d['geographic_name'] = location
        d['year'] = year
        df1 = df.loc['Nigeria', ['Series Code', year]]
        for key, value in codes.items():
            d[year + '_' + key] = df1.loc[df1['Series Code'] == value, year].values[0]
    else:
        for i in countries:
            if i.lower() in location.lower():
                d['geographic_name'] = location
                d['year'] = year
                d['country'] = i
                df1 = df.loc[i, ['Series Code', year]]
                for key, value in codes.items():
                    d[year + '_' + key] = df1.loc[df1['Series Code'] == value, year].values[0]
    return d

In [ ]:
for message in messages:
    if 'GeoLocationParser-data' in item:
        year = item['timestamp'].split('-')[0]
        GeoLocations = item['GeoLocationParser-data'][0]
        keys = GeoLocations.keys()
        for key in keys:
            regex = re.compile('Optional_NAME[\d]*')
            if key == 'Geographic_NAME':
                location = GeoLocations['Geographic_NAME']
                try:
                    to_append = lookup_loc(location, year, ids_df)
                    if to_append != {}:
                        item['GeoLocationParser-data'].append({(key + "_economic_data"): to_append})
                except:
                    print("Error: " + key + " " + location)
            elif re.match(regex, key):
                location = GeoLocations[key]
                try:
                    to_append = lookup_loc(location, year, ids_df)
                    if to_append != {}:
                        item['GeoLocationParser-data'].append({(key + "_economic_data"): to_append})
                except:
                    print("Error: " + key + " " + location)
            else:
                continue

### Dataset 3:  Enron Email Corpus
#### <font color = darkblue>MIME Top-Level: message<br>MIME Sub-level: message</font>   
- Purpose: 
- Sources:
    - Archived files: http://www.enron-mail.com/
    - Compressed Data: https://www.cs.cmu.edu/~./enron/

#### <font color = darkblue>Enron New Features</font>
1. 
2. 
3. 

## Export Updated JSON File with Additional Datasets